In [18]:
import os
import pyfftw
import numpy as np

from drdmannturb.fluctuation_generation.covariance_kernels import Covariance

In [19]:
param_sets = {
        "figure2_a_eq15": {
        "sigma2": 2.0,  # m2/s2
        "L_2d": 15_000.0,  # m
        "psi": np.deg2rad(45.0),  # degrees
        "z_i": 500.0,  # m
        "L1_factor": 40,
        "L2_factor": 5,
        "N1": 10,
        "N2": 7,
        "equation": "eq15",
    },
    "figure2_b_eq16": {
        "sigma2": 2.0,  # m2/s2
        "L_2d": 15_000.0,  # m
        "psi": np.deg2rad(45.0),  # degrees
        "z_i": 500.0,  # m
        "L1_factor": 1,
        "L2_factor": 0.125,
        "N1": 10,
        "N2": 7,
        "equation": "eq16",
    },
    "figure3_standard_eq14": {
        "sigma2": 0.6,  # m2/s2
        "L_2d": 15_000.0,  # m
        "psi": np.deg2rad(45.0),  # degrees
        "z_i": 500.0,  # m
        "L1_factor": 4,
        "L2_factor": 1,
        "N1": 10,
        "N2": 7,
        "equation": "eq14",
    },
    "figure3_standard_eq15": {
        "sigma2": 0.6,  # m2/s2
        "L_2d": 15_000.0,  # m
        "psi": np.deg2rad(45.0),  # degrees
        "z_i": 500.0,  # m
        "L1_factor": 4,
        "L2_factor": 1,
        "N1": 10,
        "N2": 7,
        "equation": "eq15",
    },
}

In [27]:
config = param_sets["figure2_a_eq15"]

_sigma2 = config["sigma2"]
_L_2d = config["L_2d"]
_psi = config["psi"]
_z_i = config["z_i"]
_L1_factor = config["L1_factor"]
_L2_factor = config["L2_factor"]
_N1 = config["N1"]
_N2 = config["N2"]

# Calculate 
grid_dimensions = np.array([
    _L_2d * _L1_factor, 
    _L_2d * _L2_factor, 
])
grid_levels = np.array([_N1, _N2])

# NOTHING ABOVE THIS LINE OF SIGNIFICANT IMPORTANCE!
--------------------------------------------------

In [144]:
#######################################################################################
# Sampling methods

class Sampling_method_base:

    def __init__(self, RandomField):

        self.L, self.Nd, self.ndim = (
            RandomField.L,
            RandomField.ext_grid_shape,
            RandomField.ndim,
        )
        self.DomainSlice = RandomField.DomainSlice


class Sampling_method_freq(Sampling_method_base):

    def __init__(self, RandomField):
        super().__init__(RandomField)
        L, Nd, d = self.L, self.Nd, self.ndim
        
        print("L: ", L)
        print("Nd: ", Nd)
        print("d: ", d)

        self.Frequencies = [(2 * np.pi / L[j]) * (Nd[j] * np.fft.fftfreq(Nd[j])) for j in range(d)]
        print("REACHED HERE")
        self.TransformNorm = np.sqrt(L.prod())
        self.Spectrum = RandomField.Covariance.precompute_Spectrum(self.Frequencies)


class Sampling_VF_FFTW(Sampling_method_freq):

    def __init__(self, RandomField):
        super().__init__(RandomField)

        # WARN: User might have OMP_NUM_THREADS set to something invalid here
        n_cpu = int(os.environ.get("OMP_NUM_THREADS", 4))

        shpR = RandomField.ext_grid_shape
        shpC = shpR.copy()
        shpC[-1] = int(shpC[-1] // 2) + 1
        axes = np.arange(self.ndim)

        print("shpR", shpR)
        print("shpC", shpC)
        print("axes", axes)

        flags = ("FFTW_MEASURE", "FFTW_DESTROY_INPUT", "FFTW_UNALIGNED")
        self.fft_x = pyfftw.empty_aligned(shpR, dtype="float32")
        self.fft_y = pyfftw.empty_aligned(shpC, dtype="complex64")

        print("self.fft_x.shape", self.fft_x.shape)
        print("self.fft_y.shape", self.fft_y.shape)

        self.fft_plan = pyfftw.FFTW(
            self.fft_x,
            self.fft_y,
            axes=axes,
            direction="FFTW_FORWARD",
            flags=flags,
            threads=n_cpu,
        )
        self.ifft_plan = pyfftw.FFTW(
            self.fft_y,
            self.fft_x,
            axes=axes,
            direction="FFTW_BACKWARD",
            flags=flags,
            threads=n_cpu,
        )
        self.Spectrum_half = self.Spectrum[..., : shpC[-1]] * np.sqrt(self.Nd.prod())

        print("self.Spectrum_half.shape", self.Spectrum_half.shape)

        self.hat_noise = np.stack([np.zeros(shpC, dtype="complex64") for _ in range(3)], axis=-1)

        print("self.hat_noise.shape", self.hat_noise.shape)

        self.shpC = shpC

    def __call__(self, noise):
        tmp = np.zeros(noise.shape)

        print("__call__: noise.shape", noise.shape)
        print("__call__: tmp.shape", tmp.shape)

        print("__call__: self.fft_x.shape", self.fft_x.shape)
        print("__call__: self.fft_y.shape", self.fft_y.shape)

        print("Iterating over noise.shape[-1] = ", noise.shape[-1])

        for i in range(noise.shape[-1]):
            self.fft_x[:] = noise[..., i]
            self.fft_plan()
            self.hat_noise[..., i] = self.fft_y[:]

        # TODO: Check this einsum.

        print("__call__: self.Spectrum_half.shape", self.Spectrum_half.shape)
        print("__call__: self.hat_noise.shape", self.hat_noise.shape)

        self.hat_noise = np.einsum("kl...,...l->...k", self.Spectrum_half, self.hat_noise)

        print("AFTER EINSUM __call__: self.hat_noise.shape", self.hat_noise.shape)

        for i in range(noise.shape[-1]):
            self.fft_y[:] = self.hat_noise[..., i]
            self.ifft_plan()
            tmp[..., i] = self.fft_x[:]
        return tmp[self.DomainSlice] / self.TransformNorm


In [145]:
class LowFreqCovariance(Covariance):
    def __init__(self, sigma2, L2d, psi, z_i):
        super().__init__()

        if self.ndim != 2:
            raise ValueError("ndim must be 2 for MannSyed LowFreq covariance.")
        self.ndim = 2

        self.sigma2 = sigma2
        self.L2d = L2d
        self.psi = psi
        self.z_i = z_i

    def precompute_Spectrum(self, Frequencies):

        Nd = [Frequencies[j].size for j in range(self.ndim)]

        SqrtSpectralTens = np.tile(np.zeros(Nd), (2, 2, 1, 1, 1))

        return SqrtSpectralTens * 1j

In [146]:
#######################################################################################
# Random field

class GaussianRandomField:

    def __init__(self,
        Covariance,
        grid_level,
        grid_shape,
        grid_dimensions,
    ):

        self.ndim = 2
        self.sampling_method = "fft"

        assert len(grid_dimensions) == 2
        assert len(grid_level) == 2

        h = grid_dimensions / (2**grid_level + 1)
        # TODO: self.grid_shape should probably just be set to grid_shape
        self.grid_shape = grid_shape[:2]

        self.L = h * self.grid_shape

        ### Extended window
        N_margin = 0
        self.ext_grid_shape = self.grid_shape

        # if self.ext_grid_shape == self.grid_shape:
            # print("\tNOTE: grid_shape and ext_grid_shape are the same")

        self.nvoxels = self.ext_grid_shape.prod()
        self.DomainSlice = tuple([slice(N_margin, N_margin + self.grid_shape[j]) for j in range(self.ndim)])

        print("Constructed up to Domain Slice")

        ### Covariance kernel
        self.Covariance = Covariance

        self.prng = np.random.RandomState()
        self.noise_std = np.sqrt(np.prod(h))
        self.distribution = self.prng.normal

        print("CONSTRUCTED UP TO CORRELATE AND SAMPLING METHOD")

        ### Sampling method
        self.Correlate = Sampling_VF_FFTW(self)
        self.method = "vf_fftw"

    def reseed(self, seed=None):

        if seed is not None:
            self.prng.seed(seed)
        else:
            self.prng.seed()

    def _sample_noise(self, grid_shape):

        if grid_shape is None:
            noise = self.distribution(0, 1, self.ext_grid_shape)
        else:
            noise = self.distribution(0, 1, grid_shape)

        noise *= self.noise_std

        print("sample_noise: noise.shape", noise.shape)
        return noise
    
    def sample(self, noise = None) -> np.ndarray:

        if not hasattr(self, "Correlate"):
            raise Exception("Sampling method not set.")

        if noise is None:
            noise = self._sample_noise()

        field = self.Correlate(noise)

        print("sample: field.shape", field.shape)

        return field


class VectorGaussianRandomField(GaussianRandomField):

    def __init__(self, vdim = 2, **kwargs):
        super().__init__(**kwargs)
        self.vdim = vdim
        self.DomainSlice = tuple(list(self.DomainSlice) + [slice(None, None, None)])

        self.Correlate.DomainSlice = self.DomainSlice


The following are done in the FluctuationFieldGenerator constructor:

1. Define covariance object,
2. Make GRF object,
  - This hides the construction of the sampling method, which is fixed to VF_FFTW
3. Sample field

Then, to generate the field, in `generate(...)`, the following happens:
1. `_generate_block(...)` is called, which samples the field
2. `_normalize_block(...)` is called, which normalizes the field
3. `total_fluctuation` is updated by concatenating the new block onto it


In `_generate_block(...)`, the following happens:

- If there is not already a noise sample, then one is created. Else, the noise is resampled for the
  next block.
- Then, using what is now `self.noise`, the field is sampled.
- NOT NEEDED: now, there is some conditional work with the blend number/region, which we're not doing
  here.




In [147]:
config = param_sets["figure2_a_eq15"]

_sigma2 = config["sigma2"]
_L_2d = config["L_2d"]
_psi = config["psi"]
_z_i = config["z_i"]
_L1_factor = config["L1_factor"]
_L2_factor = config["L2_factor"]
_N1 = config["N1"]
_N2 = config["N2"]

# Calculate 
grid_dimensions = np.array([
    _L_2d * _L1_factor, 
    _L_2d * _L2_factor, 
])
grid_levels = np.array([_N1, _N2])

In [148]:
grid_node_counts = 2**grid_levels + 1
grid_spacings = [dim / size for dim, size in zip(grid_dimensions, grid_node_counts)]

Nx, Ny = grid_node_counts
dx, dy = grid_spacings

# TODO: check -- this is weird
n_buffer = np.ceil( (3 * _L_2d) / dx )
n_margins = [
    np.ceil(_L_2d /dy),
]

wind_shape = [0, Ny, 2]

buffer_extension = 2 * n_buffer
margin_extension = [2 * margin for margin in n_margins]

noise_shape = np.array([
    Nx + buffer_extension,
    Ny + margin_extension[0],
    2
])


In [150]:
lfc = LowFreqCovariance(
    _sigma2,
    _L_2d,
    _psi,
    _z_i,
)

print("\nLowFreqCovariance object created \n")

grf = GaussianRandomField(
    # ndim is fixed to 2
    lfc,
    grid_level=grid_levels,
    grid_dimensions=grid_dimensions,
    # sampling_method is fixed to "vf_fftw"
    grid_shape=noise_shape[:-1],
)


LowFreqCovariance object created 

Constructed up to Domain Slice
CONSTRUCTED UP TO CORRELATE AND SAMPLING METHOD
L:  [690146.34146341 105232.55813953]
Nd:  [1179.  181.]
d:  2


ValueError: n should be an integer

In [151]:
wind_block = grf.sample()

TypeError: _sample_noise() missing 1 required positional argument: 'grid_shape'